In [1]:
# -*- coding: utf-8 -*-
import os
import sys
import pandas as pd
import re
import numpy as np
path = os.path.join(os.path.dirname(os.path.abspath("__file__")), 'keras_yolo3/')
sys.path.append(path)


In [2]:
"""
Class definition of YOLO_v3 style detection model on image and video
"""
import colorsys
from timeit import default_timer as timer

import numpy as np
from keras import backend as K
from keras.models import load_model
from keras.layers import Input
from PIL import Image, ImageFont, ImageDraw

from yolo3.model import yolo_eval, yolo_body, tiny_yolo_body
from yolo3.utils import letterbox_image
from keras.utils import multi_gpu_model

import glob
from PIL import Image

Using TensorFlow backend.


In [3]:
class MyYOLO(object):
    yolo_path = "keras_yolo3/"
    _defaults = {
        "model_path": yolo_path + 'model_data/yolo.h5',
        "anchors_path": yolo_path + 'model_data/yolo_anchors.txt',
        "classes_path": yolo_path + 'model_data/coco_classes.txt',
        "score" : 0.3,
        "iou" : 0.45,
        "model_image_size" : (416, 416),
        "gpu_num" : 1,
    }

    @classmethod
    def get_defaults(cls, n):
        if n in cls._defaults:
            return cls._defaults[n]
        else:
            return "Unrecognized attribute name '" + n + "'"

    def __init__(self, **kwargs):
        self.__dict__.update(self._defaults) # set up default values
        self.__dict__.update(kwargs) # and update with user overrides
        self.class_names = self._get_class()
        self.anchors = self._get_anchors()
        self.sess = K.get_session()
        self.boxes, self.scores, self.classes = self.generate()

    def _get_class(self):
        classes_path = os.path.expanduser(self.classes_path)
        with open(classes_path) as f:
            class_names = f.readlines()
        class_names = [c.strip() for c in class_names]
        return class_names

    def _get_anchors(self):
        anchors_path = os.path.expanduser(self.anchors_path)
        with open(anchors_path) as f:
            anchors = f.readline()
        anchors = [float(x) for x in anchors.split(',')]
        return np.array(anchors).reshape(-1, 2)

    def generate(self):
        model_path = os.path.expanduser(self.model_path)
        assert model_path.endswith('.h5'), 'Keras model or weights must be a .h5 file.'

        # Load model, or construct model and load weights.
        num_anchors = len(self.anchors)
        num_classes = len(self.class_names)
        is_tiny_version = num_anchors==6 # default setting
        try:
            self.yolo_model = load_model(model_path, compile=False)
        except:
            self.yolo_model = tiny_yolo_body(Input(shape=(None,None,3)), num_anchors//2, num_classes) \
                if is_tiny_version else yolo_body(Input(shape=(None,None,3)), num_anchors//3, num_classes)
            self.yolo_model.load_weights(self.model_path) # make sure model, anchors and classes match
        else:
            assert self.yolo_model.layers[-1].output_shape[-1] == \
                num_anchors/len(self.yolo_model.output) * (num_classes + 5), \
                'Mismatch between model and given anchor and class sizes'

        print('{} model, anchors, and classes loaded.'.format(model_path))

        # Generate colors for drawing bounding boxes.
        hsv_tuples = [(x / len(self.class_names), 1., 1.)
                      for x in range(len(self.class_names))]
        self.colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
        self.colors = list(
            map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
                self.colors))
        np.random.seed(10101)  # Fixed seed for consistent colors across runs.
        np.random.shuffle(self.colors)  # Shuffle colors to decorrelate adjacent classes.
        np.random.seed(None)  # Reset seed to default.

        # Generate output tensor targets for filtered bounding boxes.
        self.input_image_shape = K.placeholder(shape=(2, ))
        if self.gpu_num>=2:
            self.yolo_model = multi_gpu_model(self.yolo_model, gpus=self.gpu_num)
        boxes, scores, classes = yolo_eval(self.yolo_model.output, self.anchors,
                len(self.class_names), self.input_image_shape,
                score_threshold=self.score, iou_threshold=self.iou)
        return boxes, scores, classes

    def detect_image(self, image):
        start = timer()
        if self.model_image_size != (None, None):
            assert self.model_image_size[0]%32 == 0, 'Multiples of 32 required'
            assert self.model_image_size[1]%32 == 0, 'Multiples of 32 required'
            print(image)
            
            boxed_image = letterbox_image(image, tuple(reversed(self.model_image_size)))

        else:
            new_image_size = (image.width - (image.width % 32),
                              image.height - (image.height % 32))
            boxed_image = letterbox_image(image, new_image_size)
        image_data = np.array(boxed_image, dtype='float32')

        print(image_data.shape)
        image_data /= 255.
        image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

        out_boxes, out_scores, out_classes = self.sess.run(
            [self.boxes, self.scores, self.classes],
            feed_dict={
                self.yolo_model.input: image_data,
                self.input_image_shape: [image.size[1], image.size[0]],
                K.learning_phase(): 0
            })

        print('Found {} boxes for {}'.format(len(out_boxes), 'img'))

        font = ImageFont.truetype(font= self.yolo_path + 'font/FiraMono-Medium.otf',
                    size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
        thickness = (image.size[0] + image.size[1]) // 300

        for i, c in reversed(list(enumerate(out_classes))):
            predicted_class = self.class_names[c]
            box = out_boxes[i]
            score = out_scores[i]

            label = '{} {:.2f}'.format(predicted_class, score)
            draw = ImageDraw.Draw(image)
            label_size = draw.textsize(label, font)

            top, left, bottom, right = box
            top = max(0, np.floor(top + 0.5).astype('int32'))
            left = max(0, np.floor(left + 0.5).astype('int32'))
            bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
            right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
            print(label, (left, top), (right, bottom))

            if top - label_size[1] >= 0:
                text_origin = np.array([left, top - label_size[1]])
            else:
                text_origin = np.array([left, top + 1])

            # My kingdom for a good redistributable image drawing library.
            for i in range(thickness):
                draw.rectangle(
                    [left + i, top + i, right - i, bottom - i],
                    outline=self.colors[c])
            draw.rectangle(
                [tuple(text_origin), tuple(text_origin + label_size)],
                fill=self.colors[c])
            draw.text(text_origin, label, fill=(0, 0, 0), font=font)
            del draw

        end = timer()
        print(end - start)
        return image, out_boxes, out_scores, out_classes

    def close_session(self):
        self.sess.close()



In [8]:
myYolo= MyYOLO()


Instructions for updating:
Colocations handled automatically by placer.
keras_yolo3/model_data/yolo.h5 model, anchors, and classes loaded.


In [47]:
columns = ["lu_x","lu_y","ru_x","ru_y","ld_x","ld_y","rd_x","rd_y"]
df_concat = pd.Series([0,0,0,0,0,0,0,0],index = columns)
df_resize_concat = pd.Series([0,0,0,0,0,0,0,0],index = columns)
date = 20190125
excel_df = pd.read_excel(f'./{date}/{date}list.xlsx',names=columns, index=0)

df_na = (excel_df.iloc[1:,:]).dropna()
df = df_na[1:]
img_path = f"/Users/naka345/Desktop/deeplearning/number_plate/{date}/{date}img"
output_path = "/Users/naka345/Desktop/deeplearning/number_plate/output/car/"
output_csv_path = "/Users/naka345/Desktop/deeplearning/number_plate/output/csv/"
train_csv_path = "/Users/naka345/Desktop/deeplearning/number_plate/output/train/"
ls = glob.glob(img_path + "/*.JPG")
c=0
for path in ls:
    file_name = path.split('/')[-1]
    file_num = re.sub(r'\D', '', file_name)
    vertex = df.loc[int(file_num)]
    print(file_name)
    image = Image.open(path)
    image = image.rotate(270, expand=True)
    image_size = image.size
    org_image = image.copy()
    image, out_boxes, out_scores, out_classes = myYolo.detect_image(image)
    image.save(output_path + '../' + file_name)
    predict_pos = choice_box(vertex, out_boxes, out_scores, out_classes, image_size)
    if predict_pos is None:
        del image,org_image,vertex
        continue
    plate_npx=np.array([vertex["lu_x"],vertex["ld_x"],vertex["ru_x"],vertex["rd_x"]])
    plate_npy=np.array([vertex["lu_y"],vertex["ld_y"],vertex["ru_y"],vertex["rd_y"]])
    # one car
    one_car_img = org_image.crop((predict_pos['left'], predict_pos['top'], predict_pos['right'], predict_pos['bottom']))
    one_car_img.save(output_path + file_name)
    moved_vertex = number_plate_crop(one_car_img, vertex, predict_pos, file_name)

    df_concat = pd.concat([df_concat, moved_vertex],axis=1)
    # detect_char_on_plate()
    resized_df = resize_image(one_car_img,file_num,moved_vertex)
    df_resize_concat = pd.concat([df_resize_concat, resized_df],axis=1)
    del image,org_image,vertex


df_T=df_concat.T
df_T[1:].to_csv(f'{output_csv_path}{date}.csv')
df_concat_T=df_resize_concat.T
df_concat_T[1:].to_csv(f'{train_csv_path}{date}.csv')

IMG_4474.JPG
<PIL.Image.Image image mode=RGB size=3024x4032 at 0x129AFD208>
(416, 416, 3)
Found 2 boxes for img
car 0.77 (14, 0) (738, 417)
car 0.99 (824, 371) (3007, 2833)
1.5275399670026673
lu_x     720
lu_y    1585
ru_x    1323
ru_y    1806
ld_x     672
ld_y    1825
rd_x    1242
rd_y    2050
Name: 4474, dtype: object
IMG_4449.JPG
<PIL.Image.Image image mode=RGB size=3024x4032 at 0x129D2C7F0>
(416, 416, 3)
Found 4 boxes for img
truck 0.37 (2559, 437) (3011, 1264)
car 0.67 (2559, 437) (3011, 1264)
car 1.00 (20, 231) (1309, 1260)
car 1.00 (379, 260) (2781, 1985)
1.5639528079991578
lu_x     337
lu_y    1044
ru_x     643
ru_y    1134
ld_x     331
ld_y    1221
rd_x     628
rd_y    1317
Name: 4449, dtype: object
IMG_4462.JPG
<PIL.Image.Image image mode=RGB size=3024x4032 at 0x129D2C7F0>
(416, 416, 3)
Found 2 boxes for img
car 0.97 (8, 15) (1250, 1018)
car 0.97 (917, 16) (3024, 2045)
1.5634035679977387
lu_x     400
lu_y    1226
ru_x     778
ru_y    1361
ld_x     382
ld_y    1436
rd_x     74

IMG_4441.JPG
<PIL.Image.Image image mode=RGB size=3024x4032 at 0x129AA5710>
(416, 416, 3)
Found 2 boxes for img
car 0.98 (556, 0) (2999, 1842)
car 1.00 (0, 94) (1358, 1190)
1.5248671859990282
lu_x     436
lu_y    1118
ru_x     724
ru_y    1214
ld_x     416
ld_y    1290
rd_x     700
rd_y    1392
Name: 4441, dtype: object
IMG_4469.JPG
<PIL.Image.Image image mode=RGB size=3024x4032 at 0x129D2BCC0>
(416, 416, 3)
Found 2 boxes for img
car 0.94 (0, 0) (2447, 1579)
car 0.96 (825, 87) (3003, 2556)
1.5349999120007851
lu_x     601
lu_y    1497
ru_x    1079
ru_y    1673
ld_x     571
ld_y    1733
rd_x    1031
rd_y    1903
Name: 4469, dtype: object
IMG_4468.JPG
<PIL.Image.Image image mode=RGB size=3024x4032 at 0x12A3042E8>
(416, 416, 3)
Found 2 boxes for img
car 0.81 (141, 18) (2254, 1619)
car 0.99 (1295, 351) (2976, 2637)
1.5317631229991093
lu_x     640
lu_y    1544
ru_x    1152
ru_y    1722
ld_x     582
ld_y    1757
rd_x    1063
rd_y    1950
Name: 4468, dtype: object
IMG_4508.JPG
<PIL.Image.Image

In [4]:
def choice_box(vertex, out_boxes, out_scores, out_classes, image_size):
    predict_pos=[]
    x=[vertex["lu_x"],vertex["ld_x"],vertex["ru_x"],vertex["rd_x"]]
    y=[vertex["lu_y"],vertex["ld_y"],vertex["ru_y"],vertex["rd_y"]]

    for (out_box, out_score, out_class) in zip(out_boxes,out_scores,out_classes):
        pos_dict = created_boxes_vertex_dict(out_box, image_size)
        x_pos = xpos_in_box(pos_dict, x)
        y_pos = ypos_in_box(pos_dict, y)

        if x_pos and y_pos:
            pos_dict.update({"score":out_score})
            predict_pos.append(pos_dict)
    if len(predict_pos) >= 2:
        max_score=0
        for target_box in predict_pos:
            if target_box["score"]>max_score:
                max_score = target_box["score"]
                return_box = target_box
            else:
                return_box = None
        return return_box
    elif len(predict_pos) == 0:
        return None
    return predict_pos[0]

In [5]:
def created_boxes_vertex_dict(out_box, size):
    top, left, bottom, right = out_box
    top = max(0, np.floor(top + 0.5).astype('int32'))
    left = max(0, np.floor(left + 0.5).astype('int32'))
    bottom = min(size[1], np.floor(bottom + 0.5).astype('int32'))
    right = min(size[0], np.floor(right + 0.5).astype('int32'))
    box = [top, left, bottom, right]
    
    pos_dict={}
    pos_name = ["top", "left", "bottom", "right"]
    for pos,name in zip(box, pos_name):
        pos_dict[name] = pos
    return pos_dict

def xpos_in_box(pos_dict, arr):
    for x_pos in arr:
        if (pos_dict["left"] < x_pos < pos_dict["right"]) == False:
            return False
    return True
    
def ypos_in_box(pos_dict, arr):
    for y_pos in arr:
        if (pos_dict["top"] < y_pos < pos_dict["bottom"]) == False:
            return False
    return True

In [6]:
'''   
        print(out_boxes.shape)
        out_boxes=np.append(out_boxes,np.array([[1597, 1299, 1757, 1542]]),axis=0)
        out_scores=np.append(out_scores,np.array([0.5]),axis=0)
        out_classes=np.append(out_classes,np.array([2]),axis=0)
'''

'   \n        print(out_boxes.shape)\n        out_boxes=np.append(out_boxes,np.array([[1597, 1299, 1757, 1542]]),axis=0)\n        out_scores=np.append(out_scores,np.array([0.5]),axis=0)\n        out_classes=np.append(out_classes,np.array([2]),axis=0)\n'

In [7]:
def move_and_update(vertex, corner_name, position):
    move_pos = vertex[corner_name] - position
    vertex.at[corner_name] = move_pos
    return move_pos, vertex
    
def number_plate_crop(image, vertex, predict_pos, file_name):
    import cv2
    import numpy as np
    tmp_image = np.asarray(image)
    ocv_image = tmp_image[:, :, ::-1].copy()
    
    move_lux, vertex = move_and_update(vertex, "lu_x", predict_pos["left"])
    move_ldx, vertex = move_and_update(vertex, "ld_x", predict_pos["left"])
    move_rux, vertex = move_and_update(vertex, "ru_x", predict_pos["left"])
    move_rdx, vertex = move_and_update(vertex, "rd_x", predict_pos["left"])
    move_luy, vertex = move_and_update(vertex, "lu_y", predict_pos["top"])
    move_ldy, vertex = move_and_update(vertex, "ld_y", predict_pos["top"])
    move_ruy, vertex = move_and_update(vertex, "ru_y", predict_pos["top"])
    move_rdy, vertex = move_and_update(vertex, "rd_y", predict_pos["top"])
    
    pts1 = np.float32([[move_lux,move_luy],[move_rux,move_ruy],[move_ldx,move_ldy],[move_rdx,move_rdy]])
    pts2 = np.float32([[0,0],[640,0],[0,320],[640,320]])
    #透視変換
    M = cv2.getPerspectiveTransform(pts1,pts2)
    rst = cv2.warpPerspective(ocv_image,M,(640,320))
    cv2.imwrite('/Users/naka345/Desktop/deeplearning/number_plate/output/number/'+file_name,rst)
    
    return vertex

In [24]:
df

,lu_x,lu_y,ru_x,ru_y,ld_x,ld_y,rd_x,rd_y
4422,1316,1517,1556,1581,1296,1649,1539,1718
4423,325,1027,586,1125,319,1192,578,1296
4424,1192,1578,1452,1652,1180,1726,1432,1808
4427,1296,1514,1592,1608,1270,1672,1560,1768
4430,1186,1658,1478,1760,1174,1814,1456,1920
4431,1047,1671,1347,1800,1026,1866,1332,1995
4432,1116,1428,1450,1530,1092,1618,1418,1724
4433,1294,1680,1684,1844,1274,1864,1644,2040
4435,1490,1812,1966,2010,1446,2034,1908,2244
4436,691,1704,1163,1860,663,1922,1119,2084


In [40]:
def resize_image(read_image,file_num, csv_df):
    print(csv_df)
    img_height, img_width = read_image.size


    width = 200
    height = 200
    size = (height, width)
    resize_image=read_image.resize(size, Image.LANCZOS)

    resized_df = csv_df.copy()

    zip_ratio_w = img_width/width
    zip_ratio_h = img_height/height

    resized_df.iloc[::2] /= zip_ratio_w
    resized_df.iloc[1::2] /= zip_ratio_h

    resize_image.save(f"./output/train/train_{file_num}.JPG")

    return resized_df